In [ ]:
#https://github.com/MIT-LCP/mimic-code/blob/master/concepts/severityscores/oasis.sql

In [ ]:
import os, re
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [ ]:
# Generated by MIMIC_001_OASIS_raw.ipynb
oasis_raw = pd.read_csv('data/MIMIC_001_OASIS_raw.csv')

In [ ]:
# Generated by ?
chart_events = pd.read_csv('data/MIMIC_001_chartevents.csv',
                         usecols=['SUBJECT_ID', 'HADM_ID', 'Mech_Vent'])

In [ ]:
mv_events = chart_events[chart_events.Mech_Vent==1]
mv_events.shape

In [ ]:
mv_events.head(3)

In [ ]:
oasis_raw.head(3)

In [ ]:
events_raw=oasis_raw.groupby(['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los', 'total_uop', 'Oasis'])['Oasis_Value'].agg(max_val='max', min_val='min').reset_index()

In [ ]:
events_raw.Oasis.unique()

In [ ]:
HR_events = events_raw[events_raw.Oasis=='HR']
GCS_events = events_raw[events_raw.Oasis=='GCS']
RR_events = events_raw[events_raw.Oasis=='RR']
temp_events = events_raw[events_raw.Oasis=='temp']
uop_events = events_raw[events_raw.Oasis=='uop']
MAP_events = events_raw[events_raw.Oasis=='MAP']

In [ ]:
HR_events['HR_max']=HR_events['max_val']
HR_events.pop('max_val')
HR_events['HR_min']=HR_events['min_val']
HR_events.pop('min_val')
HR_events.pop('Oasis')
print(HR_events.shape)

GCS_events['GCS_min']=GCS_events['min_val']
GCS_events.pop('min_val')
GCS_events.pop('max_val')
GCS_events.pop('Oasis')
print(GCS_events.shape)

RR_events['RR_min']=RR_events['min_val']
RR_events.pop('min_val')
RR_events['RR_max']=RR_events['max_val']
RR_events.pop('max_val')
RR_events.pop('Oasis')
print(RR_events.shape)

temp_events['temp_min']=temp_events['min_val']
temp_events.pop('min_val')
temp_events['temp_max']=temp_events['max_val']
temp_events.pop('max_val')
temp_events.pop('Oasis')
print(temp_events.shape)

MAP_events['MAP_min']=MAP_events['min_val']
MAP_events.pop('min_val')
MAP_events['MAP_max']=MAP_events['max_val']
MAP_events.pop('max_val')
MAP_events.pop('Oasis')
print(MAP_events.shape)

print('')

In [ ]:
HR_events

In [ ]:
oasis0 = pd.merge(HR_events, GCS_events, on=['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los', 'total_uop'], how='left')
oasis1 = pd.merge(oasis0, RR_events, on=['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los', 'total_uop'], how='left')
oasis2 = pd.merge(oasis1, temp_events, on=['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los', 'total_uop'], how='left')
oasis3 = pd.merge(oasis2, MAP_events, on=['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los', 'total_uop'], how='left')
oasis4 = pd.merge(oasis3, mv_events, on=['SUBJECT_ID', 'HADM_ID'], how='left')

In [ ]:
oasis_final = oasis4.drop_duplicates()

In [ ]:
len(oasis_final.HADM_ID.unique())

In [ ]:
oasis_final.to_csv('data/MIMIC_001_Oasis.csv', index=False)